### Neural Network Interaction

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import statsmodels.api as sm
import sklearn

In [3]:
def generateProcess(
    number : int = 5000,
    random_sate : int = 42,
    number_of_X : int = 2,
) -> np.array :
    np.random.seed(random_sate) # Set NumPy's random seed for reproducibility
    X = np.zeros(shape = (number, number_of_X))
    for i in range(number_of_X) :
        X[:int(number/2), i] = np.random.randn(int(number/2))
        # Use antithetic sampling to reduce the bias in the mean
        X[int(number/2):, i] = -X[:int(number/2), i]
    
    eps = np.zeros(shape=(number,1))
    eps[:int(number/2)] = np.random.randn(int(number/2), 1)
    eps[int(number/2):] = -eps[:int(number/2)]
    
    Y = 0
    
    for i in range(number_of_X) : 
        Y += X[:, i]
        
    Y += eps.flatten()
    
    return Y, X;

In [4]:
Y, X = generateProcess()

In [6]:
ols_results = sm.OLS(
    Y,
    sm.add_constant(X)
).fit()

In [7]:
y_ols = ols_results.predict(sm.add_constant(X))

In [8]:
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.642
Model:                            OLS   Adj. R-squared:                  0.642
Method:                 Least Squares   F-statistic:                     4479.
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        14:24:02   Log-Likelihood:                -7205.3
No. Observations:                5000   AIC:                         1.442e+04
Df Residuals:                    4997   BIC:                         1.444e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.435e-17      0.014  -9.92e-16      1.0

#### Feedforward Neural Network

compare with a feedforward neural network with no hidden layers

In [9]:
def linear_NN0_model(l1_reg = .0, num_X_input : int = 2) :
    model = Sequential()
    model.add(
        Dense(1, input_dim = num_X_input, kernel_initializer = 'normal')
    )
    model.compile(
        loss = 'mean_squared_error',
        optimizer = 'adam',
        metrics = ['mae','mse']
    )
    return model;

In [10]:
es = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 0, patience = 10)

In [11]:
linear_neural_network = KerasRegressor(
    build_fn = linear_NN0_model, 
    epochs = 40,
    batch_size = 10, 
    verbose = 0, 
    callbacks = [es]
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_16541/4154843586.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  linear_neural_network = KerasRegressor(


In [12]:
linear_neural_network.fit(X, Y)

2023-12-19 14:28:53.843477: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [13]:
print("weights: " + str(linear_neural_network.model.layers[0].get_weights()[0]))
print("bias: " + str(linear_neural_network.model.layers[0].get_weights()[1]))

weights: [[0.9890076 ]
 [0.96185046]]
bias: [-0.00270257]
